# Explicando Retrieval Augmented Generation

Neste exemplo, vamos carregar um PDF com prospecto de um fundo de investimento para utilizá-lo como base de conhecimento para tratamentos de dúvidas.

Utilizaremos os modelos da OpenAI para gerar embeddings e respostas.

Os dados vetorizados serão armazenados no Astra/Cassandra para busca baseada na similaridade de vetores (Vector Search).

## Hands-on: Setup / Configurando ambiente

Inicialmente, vamos instalar e configurar o ambiente para execução dos modelos

In [ ]:
pip install langchain llama-index openai cassio --upgrade 

In [4]:
# Carregando variáveis de ambiente
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

### Conectando ao Astra/Cassandra

Para utilizar o Cassandra como Vector DB, crie um banco de dados com suporte a vvector search em astra.datastax.com, gere um token de conexão e identifique o seu DB ID. Com isso, execute o comando abaixo para criar a conexão

In [5]:
import cassio

cassio.init(token=os.environ["ASTRA_DB_APPLICATION_TOKEN"], database_id=os.environ["ASTRA_DB_ID"])

Pronto!

A conexão com o Astra é muito simples e você já pode armazenar seus documentos.

### LangChain, Cassandra como Vector Store e OpenAI

In [6]:
# Importando o índice, separadores de texto, e auxiliares para processamendo dos documentos
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Hands-on: Embeddings

Vamos converter nossa base de documentos em embeddings. 

Se você não sabe o que são embeddings, assista aqui: https://www.youtube.com/watch?v=fvRyziDmvoA

Utilizaremos o LangChain para simplificar a leitura dos PDFs, geração de chunks e posterior utilização dos dados.

In [7]:
# Neste exemplo, vou utilizar os embeddings da OpenAI.
# Aqui, importamos o LLM e os Embeddings
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = OpenAI(temperature=0.1)
embedding_generator = OpenAIEmbeddings()

In [8]:
#Gerando um embedding para uma pergunta do usuário
emb = OpenAIEmbeddings().embed_query("Qual o risco do fundo de investimento?")
print(f"Dimensões no embedding: {len(emb)}")
print(f"5 primeiras dimensões do embedding: {emb[:5]}")


Dimensões no embedding: 1536
5 primeiras dimensões do embedding: [0.01560478775073706, -0.026357666071101002, 0.016221111894567722, -0.028770506832908865, -0.031025989467398273]


### Dados utilizados no exercício

O arquivo utilizado neste exercício está disponível no url: https://conteudos.xpi.com.br/fundos-de-investimento/ip-participacoes-ipg-fic-fia-bdr-nivel-i/

Apesar de este conteúdo estar disponível na internet, ele NÃO FOI considerado no treinamento do GPT, por isso o modelo de linguagem não é capaz de responder perguntas sobre este fundo.

Baixe o arquivo e grave no diretório "pdf"

### Definindo o Index/Tabela do Cassandra que vai armazenar os dados

In [9]:
# O LangChain possui suporte ao Cassandra, então vamos utilizá-lo
# (vamos usar o Astra, que é o Cassandra como serviço)
# Criando o índice no Astra

from langchain.vectorstores.cassandra import Cassandra
table_name = 'vs_investment_kb'
keyspace = 'demo'
file = './pdf/Lamina_12082452000149_v46.pdf'

In [ ]:
# Se necessário, execute este comando para limpar a tabela com os dados
#CassVectorStore.clear()

## Criando o index e carregando

O método IndexCreator executa a criação da tabela e vincula qual modelo de embeddings será utilizado no índice

In [10]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Cassandra,
    embedding=embedding_generator,
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=30,
    ),
    vectorstore_kwargs={
        'session': cassio.config.resolve_session(),
        'keyspace': keyspace,
        'table_name': table_name,
    },
)

In [11]:
# Carregando os dados de um fundo de investimento específico
#loader = PyPDFDirectoryLoader('./pdf')
print(f"Loading file {file}")
loader = PyPDFLoader(file)
index = index_creator.from_loaders([loader])

Loading file ./pdf/Lamina_12082452000149_v46.pdf


In [ ]:
# Carregando os dados de um fundo de investimento específico
#loader = PyPDFDirectoryLoader('./funds')
#loader = PyPDFLoader("./funds/RealInvFIM0623.pdf")
#index = index_creator.from_loaders([loader])

### Se o seu índice já estiver carregado...

... você pode referenciá-lo desta maneira.

In [ ]:
CassVectorStore = Cassandra(
    session= cassio.config.resolve_session(),
    keyspace= keyspace,
    table_name= table_name,
    embedding=embedding_generator
)

index = VectorStoreIndexWrapper(
    vectorstore=CassVectorStore
)

### Conferindo os dados gravados

Faremos uma query no Vector Store para conferir como os dados foram gravados

In [12]:
cqlSelect = f"""SELECT * FROM {keyspace}.{table_name} 
WHERE metadata_s['source'] = '{file}'
LIMIT 3;"""
rows = cassio.config.resolve_session().execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\n{"-"*50}')
    print(f'\nRow {row_i}:')
    print(f'    document_id:      {row.row_id}')
    print(f'    embedding_vector: {str(row.vector)[:64]} ...')
    print(f'    metadata_blob:    {row.metadata_s}')
    print(f'    document:         {row.body_blob} ...')

print('\n...')


--------------------------------------------------

Row 0:
    document_id:      7fa1d24ffa214b57b169eb6bcbefb25a
    embedding_vector: [-0.0006555379368364811, -0.01693853549659252, 0.000957359443418 ...
    metadata_blob:    {'page': '0.0', 'source': './pdf/Lamina_12082452000149_v46.pdf'}
    document:         3. POLÍTICA DE  INVESTIMENTOS
A política  de investimento  do FUNDO é  manter, no mínimo, 95% (noventa e cinco por cento) do seu patrimônio  líquido aplicado em cotas do IP-
PARTICIPAÇÕES INQ FUNDO DE INVESTIMENTO EM AÇÕES  BDR NÍVEL I  (“FUNDO ALVO”),  inscrito no CNPJ  sob nº 28.246.609/0001-64, 
administrado pela BNY Mellon Serviços Financeiros DTVM  S.A. e gerido pela Investidor Profissional Gestão de Recursos Ltda, bem como em outros ...

--------------------------------------------------

Row 1:
    document_id:      b3986902916d4fafae21a352540f3011
    embedding_vector: [-0.02479463256895542, 0.0013354832772165537, 0.0199969727545976 ...
    metadata_blob:    {'page': '

# Hands-on:  Q/A - Utilizando o documento para responder uma pergunta

O LangChain possui métodos que buscam documentos similares no index e compõem um prompt para gerar a resposta a uma pergunta.

Este método são o ``ìndex.query`` e ``index.query_with_sources``.

A diferença entre estes métodos é o tipo de RetrievalChain que são utilizados por cada um.


In [15]:
import langchain
langchain.debug = False
langchain.verbose = True

QUERY = "Qual o rendimento do fundo?"

print("\nResposta com o query_with_sources:")
print(index.query_with_sources(question=QUERY))


Resposta com o query_with_sources:


> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not consti

In [16]:
import langchain
langchain.debug = False
langchain.verbose = False

QUERY = "Qual o risco do fundo?"

print("Resposta com o query:")
print(index.query(question=QUERY))

print("\nResposta com o query_with_sources:")
print(index.query_with_sources(question=QUERY))

Resposta com o query:
 O BNY Mellon Serviços Financeiros classifica os fundos que administra numa escala de 1 a 5 de acordo com o risco envolvido na estratégia de investimento de cada um deles. Nessa escala, a classificação do fundo é: 4,20.

Resposta com o query_with_sources:
{'question': 'Qual o risco do fundo?', 'answer': ' O risco do fundo é classificado como 4,20 na escala de 1 a 5 de acordo com o risco envolvido na estratégia de investimento. O fundo pode aplicar em ativos no exterior até o limite de 20%, em crédito privado até o limite de 33%, em cotas de um mesmo fundo de investimento até o limite de 100% e se alavancar até o limite de 100%.\n', 'sources': './pdf/Lamina_12082452000149_v46.pdf'}


Estes métodos funcionam bem, mas na vida real precisaremos de mais flexibilidade e detalhamento no comportamento do LLM, modificando o prompt para fornecer mais instruções, ou filtrando documentos que queremos que sejam considerados.

Por isso, o melhor é usarmos as mesmas Chains utilizadas acima, mas agora podendo especificar mais detalhes. 

Vamos ver como fazer isso.



# Hands-on: Q/A passo a passo

## Buscando documentos pela similaridade

A busca pelos documentos que vão preencher o contexto pode utilizar estratégias distintas.

Iniciaremos com a busca por similaridade de vetores.

In [18]:
CassVectorStore = Cassandra(
    session= cassio.config.resolve_session(),
    keyspace= keyspace,
    table_name= table_name,
    embedding=embedding_generator
)

matchesSim = CassVectorStore.search(QUERY, search_type='similarity', k=4)
for i, doc in enumerate(matchesSim):
    print(f'[{i:2}]: "{doc.page_content}"')

[ 0]: "os riscos do FUNDO, especialmente no sentido de que  rentabilidade passada  não é garantia  de resultados  futuros e que a aplicação em fundos  não 
contam com garantia do  ADMINISTRADOR, da GESTORA, de qualquer mecanismo de seguro ou do Fundo  Garantidor de Créditos – FGC. Os  
Distribuidores são orientados a somente utilizar materiais  de venda do FUNDO previamente avaliados pelo ADMINISTRADOR ou, quando aplicável, pela 
GESTORA."
[ 1]: "garantia junto às centrais depositárias, mas pode investir  em fundos de investimento que podem estar  expostos aos riscos  decorrentes de aplicações 
em ativos que incorram  em depósito de margem de garantia. As informações apresentadas são  provenientes dos fundos  investidos geridos por 
instituições ligadas.As estratégias de investimento do fundo podem resultar  em significativas perdas patrimoniais para  seus cotistas."
[ 2]: "Operações  compromissadas  lastreadas  em  títulos públicos federais
10,64%
Derivativos 0,86%
Cotas de fundos de 

## Utilizando a Chain com prompt personalizado
Vamos usar estes documentos para preencher o prompt. Note que nos próximos passos iremos importar as Chains e também definir o prompt que queremos utilizar.

In [19]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain import PromptTemplate

In [20]:
prompt_template = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["summaries", "question"]
)

In [23]:
langchain.verbose = True

retrieverSim = CassVectorStore.as_retriever(
    search_type='similarity',
    search_kwargs={
        'k': 4,
        'filter': {"source": file}
    },
)

# Create a "RetrievalQA" chain
chainSim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverSim,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseSim = chainSim.run(QUERY)
print(responseSim)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: Qual o risco do fundo?
os riscos do FUNDO, especialmente no sentido de que  rentabilidade passada  não é garantia  de resultados  futuros e que a aplicação em fundos  não 
contam com garantia do  ADMINISTRADOR, da GESTORA, de qualquer mecanismo de seguro ou do Fundo  Garantidor de Créditos – FGC. Os  
Distribuidores são orientados a somente utilizar materiais  de venda do FUNDO previamente avaliados pelo ADMINISTRADOR ou, quando aplicável, pela 
GESTORA.

garantia junto às centrais depositárias, mas pode investir  em fundos de investimento que podem est

In [24]:
# Busca com MMR (Maximal Marginal Relevance)
retrieverMMR = CassVectorStore.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 4,
        'fetch_k': 10,
    },
    return_source_documents=True
)

# Create a "RetrievalQA" chain
chainMMR = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverMMR,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseMMR = chainMMR.run(QUERY)
print(responseMMR)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: Qual o risco do fundo?
os riscos do FUNDO, especialmente no sentido de que  rentabilidade passada  não é garantia  de resultados  futuros e que a aplicação em fundos  não 
contam com garantia do  ADMINISTRADOR, da GESTORA, de qualquer mecanismo de seguro ou do Fundo  Garantidor de Créditos – FGC. Os  
Distribuidores são orientados a somente utilizar materiais  de venda do FUNDO previamente avaliados pelo ADMINISTRADOR ou, quando aplicável, pela 
GESTORA.

Operações  compromissadas  lastreadas  em  títulos públicos federais
10,64%
Derivativos 0,86%
Cotas

# Hands-on: Memória adicionando memória ao contexto

Após a primeira interação, as interações podem demandar que o contexto da conversação seja considerado nas próximas interações.

Para isso, é necessário registrarmos as mensagens também no banco de dados

In [25]:
from langchain.memory import CassandraChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory
memory_table_name = 'vs_investment_memory'

In [26]:
def get_answer(conversation_id, q):
    prompt_template = """
    Given the following extracted parts of a long document and a question, create a final answer in a very short format. 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Answer in Portuguese.


    QUESTION: {question}
    =========
    {summaries}
    =========
    FINAL ANSWER:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["summaries", "question"]
    )

    message_history = CassandraChatMessageHistory(
        session_id=conversation_id,
        session= cassio.config.resolve_session(),
        keyspace= cassio.config.resolve_keyspace(),
        ttl_seconds=3600,
        table_name=memory_table_name
    )

    memory = ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=message_history,
        max_token_limit=50,
        buffer=""
    )

    retrieverSim = CassVectorStore.as_retriever(
        search_type='similarity_score_threshold',
        search_kwargs={
            'k': 5,
            "score_threshold": .8
        },
    )
    
    # Create a "RetrievalQA" chain
    chainSim = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retrieverSim,
        memory=memory,
        chain_type_kwargs={
            'prompt': PROMPT,
            'document_variable_name': 'summaries'
        }
    )
    
    # Run it and print results
    answer = chainSim.run(q)

    return answer

langchain.debug = False
langchain.verbose = False

res = get_answer('my_conv2',QUERY)
print("="*20)
print(f"Query: {QUERY}")
print(res)

Query: Qual o risco do fundo?
 O risco do fundo é classificado como 4,20 na escala de 1 a 5, e pode aplicar em ativos no exterior até 20%, crédito privado até 33%, cotas de um mesmo fundo de investimento até 100% e se alavancar até 100%.


In [29]:
from langchain.chains import ConversationalRetrievalChain

def get_answer(conversation_id, q):
    prompt_template = """
    Given the following extracted parts of a long document and a question, create a final answer in a very short format. 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Answer in Portuguese.


    QUESTION: {question}
    =========
    Chat History:
    ========
    {chat_history}
    ========
    {summaries}
    =========
    FINAL ANSWER:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["summaries", "question","chat_history"]
    )

    message_history = CassandraChatMessageHistory(
        session_id=conversation_id,
        session= cassio.config.resolve_session(),
        keyspace= cassio.config.resolve_keyspace(),
        ttl_seconds=3600,
        table_name=memory_table_name
    )

    memory = ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=message_history,
        max_token_limit=50,
        buffer="",
        return_messages=True,
        memory_key="chat_history"
    )

    retrieverSim = CassVectorStore.as_retriever(
        search_type='similarity_score_threshold',
        search_kwargs={
            'k': 5,
            'filter': {"source": file},
            "score_threshold": .8
        },
    )

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever = retrieverSim,
        memory=memory
    )

    answer = qa({"question": q})
    
    return answer

In [32]:
langchain.debug = False
langchain.verbose = True
res = get_answer('my_conv3',"Quanto dinheiro o fundo tem?")
print("="*20)
print(res)



> Entering new ConversationalRetrievalChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:


New lines of conversation:
Human: Qual o rendimento?
AI:  Não sei.
Human: Qual o rendimento?
AI:  O rendimento acumulado nos últimos 5 anos foi de 56,89%.
Human: Qual o risco do fundo?
AI:  O BNY Me